In [205]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score

import glob
from operator import itemgetter

from scipy.stats import hmean

In [214]:
file_names=glob.glob('../utility/different runs with different cv/*.csv')

In [197]:
tr_files = [f for f in file_names if '_train' in f]
te_files = [f for f in file_names if '_test' in f]

In [198]:
lb_false_tr_files = [f for f in tr_files if '_train_base_lb_false' in f]
cat_tr_files = [f for f in tr_files if 'train_base_cat' in f]
lb_true_tr_files = [f for f in tr_files if f not in lb_false_tr_files + cat_tr_files]

In [202]:
lb_false_te_files = [f for f in te_files if '_base_lb_false' in f]
cat_te_files = [f for f in te_files if 'base_cat' in f]
lb_true_te_files = [f for f in te_files if f not in lb_false_te_files + cat_te_files]

In [203]:
len(lb_false_te_files), len(cat_te_files), len(lb_true_te_files)

(25, 9, 25)

In [204]:
lb_false_tr_files = [pd.read_csv(f) for f in lb_false_tr_files]
lb_false_te_files = [pd.read_csv(f) for f in lb_false_te_files]

cat_tr_files = [pd.read_csv(f) for f in cat_tr_files]
cat_te_files = [pd.read_csv(f) for f in cat_te_files]

lb_true_tr_files = [pd.read_csv(f) for f in lb_true_tr_files]
lb_true_te_files = [pd.read_csv(f) for f in lb_true_te_files]

In [217]:
def getting_gm(tr_files, te_files, file_name, HM = True):
    thresh_dict = {}
    for j,f in enumerate(tr_files):
        score_dict = {}
        for i in np.arange(0, 1, 0.01):
            score_dict[i] = f1_score(f['is_promoted'], (f['Prediction_prob'] > i).map({True:1, False:0}))
        tmep = sorted(score_dict.items(), key = itemgetter(1), reverse = True)
        thresh_dict[j] = tmep[0][0]
        print(tmep[0][1])
        
    tr = pd.concat([f['Prediction_prob'] for f in tr_files], axis = 1)
    if HM:
        tr = tr.apply(lambda x: hmean(x), axis = 1)
    else:
        tr = tr.apply(lambda x: np.prod(x)**(1/len(x)), axis = 1)
    
    oof_preds = tr_files[0][['employee_id', 'is_promoted']]
    oof_preds['Prediction_prob'] = tr
    oof_preds.to_csv('../utility/OOF_Proba_'+file_name, index = False)
    score_dict = {}
    for i in np.arange(0, 1, 0.01):
        score_dict[i] = f1_score(tr_files[0]['is_promoted'], (tr >= i).map({True:1, False:0}))
    sorted_dict = sorted(score_dict.items(), key = itemgetter(1), reverse = True) 
    optimum_th = sorted_dict[0][0]
    cv_score = sorted_dict[0][1]
    print(f'cv score is {cv_score}')
    
    sub = te_files[0][['employee_id']]
    s = pd.concat([f['is_promoted_proba'] for f in te_files], axis = 1)
    if HM:
        sub['is_promoted'] = (s.apply(lambda x: hmean(x), axis = 1) > optimum_th).map({True:1, False: 0})
    else:
        sub['is_promoted'] = (s.apply(lambda x: np.prod(x)**(1/len(x)), axis = 1) > optimum_th).map({True:1, False: 0})
    sub.to_csv('../output/'+file_name, index = False)
    if HM:
        sub['is_promoted_proba'] = s.apply(lambda x: hmean(x), axis = 1)
    else:
        sub['is_promoted_proba'] = s.apply(lambda x: np.prod(x)**(1/len(x)), axis = 1)
    sub[['employee_id', 'is_promoted_proba']].to_csv('../utility/Proba_'+file_name, index = False)

In [218]:
getting_gm(lb_true_tr_files, lb_true_te_files, 'lb_true_lgb_5x5.csv')

0.5196812364163246
0.5169593779326987
0.5178621659634317
0.5223300970873787
0.5207100591715976
0.515645894624231
0.5189787923814669
0.5188692382676506
0.5191310441485635
0.5201845444059977
0.5199676200755532
0.5173340880147164
0.5142528073711488
0.5159208865198485
0.5194733194733194
0.5184094256259205
0.5169360505973295
0.5187552097804946
0.5153132902500702
0.5179148517633835
0.5201805992611849
0.5193370165745855
0.5171270718232044
0.5163323782234956
0.5189247024207571


/Users/abk0005/Competitions/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cv score is 0.5207209577687146


In [219]:
getting_gm(lb_false_tr_files, lb_false_te_files, 'lb_false_lgb_5x5.csv')

0.5267552182163188
0.5248893063196028
0.5249095463401058
0.5254967347505906
0.5267489711934156
0.5255674049767568
0.5268169724128274
0.5302972802024035
0.5261945883707542
0.5256285595221558
0.5277445109780439
0.5281104692123432
0.5307092854312508
0.5259496312786976
0.5243851604835348
0.5271112289539971
0.526211671612265
0.5293889678336131
0.5261368465788355
0.5254955570745045
0.5296453512255237
0.5254189944134079
0.5269768769626035
0.527094258437542
0.5301650165016502


/Users/abk0005/Competitions/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cv score is 0.5299167877427025


In [220]:
getting_gm(cat_tr_files, cat_te_files, 'cat_3x3.csv')

0.5231255386383223
0.5224849895846096
0.5232753785754347
0.5313152400835074
0.5266666666666667
0.523783185840708
0.5308528756714267
0.5232497387669801
0.5291902071563088


/Users/abk0005/Competitions/env/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cv score is 0.5273545203889131


In [221]:
oof_model1 = pd.read_csv('../utility/OOF_Proba_lb_false_lgb_5x5.csv')
oof_model2 = pd.read_csv('../utility/OOF_Proba_lb_true_lgb_5x5.csv')
oof_model3 = pd.read_csv('../utility/OOF_Proba_cat_3x3.csv')

test_model1 = pd.read_csv('../utility/Proba_lb_true_lgb_5x5.csv')
test_model2 = pd.read_csv('../utility/Proba_lb_false_lgb_5x5.csv')
test_model3 = pd.read_csv('../utility/Proba_cat_3x3.csv')

In [242]:
oof_preds = pd.concat([oof_model1['Prediction_prob'], oof_model2['Prediction_prob'], oof_model3['Prediction_prob']],
                      axis = 1)
tr = oof_preds.apply(lambda x: hmean(x), axis = 1)
tr.shape

(54802,)

In [243]:
preds = pd.concat([test_model1['is_promoted_proba'], test_model2['is_promoted_proba'], test_model3['is_promoted_proba']],
                      axis = 1)
preds = preds.apply(lambda x: hmean(x), axis = 1)
preds.shape

(23490,)

In [239]:
score_dict = {}
for i in np.arange(0, 1, 0.01):
    score_dict[i] = f1_score(oof_model1['is_promoted'], (tr >= i).map({True:1, False:0}))
sorted_dict = sorted(score_dict.items(), key = itemgetter(1), reverse = True) 
optimum_th = sorted_dict[0][0]
cv_score = sorted_dict[0][1]

In [237]:
optimum_th, cv_score

(0.27, 0.5297961317302666)

In [244]:
sub = test_model1[['employee_id']]
sub['is_promoted'] = (preds > 0.27).map({True:1, False: 0})
sub['is_promoted'].mean()

0.054235845040442744

In [247]:
sub.to_csv('../output/Final_submission_wo_etc.csv', index = False)